# Evaluation
This notebook contains code to reproduce the results presented in the Energy Informatics Open Journal. 

## Testing

### Redundancy

In [ ]:
import random
import warnings
import numpy as np
import pandapower
from pathlib import Path as p

from communication.network import CommNetwork
from cyber.analysis import Analyzer
from attackers.random_attacker import RandomAttacker

seed = 42
np.random.seed(seed); random.seed(seed)
print(f"Seed: {seed}")

with warnings.catch_warnings():
    warnings.filterwarnings(action="ignore", category=FutureWarning)
    grid=pandapower.networks.create_cigre_network_mv(with_der="all")
    print(grid)
    spec_path = p.cwd() / "specifications" / "Default_specifications.json"
    network = CommNetwork(n_devices=3, n_entrypoints=1, children_per_parent=2, child_no_deviation=0, sibling_to_sibling_comm=None,
                          network_specs=spec_path, grid=grid)
print(CommNetwork.show_tree(network.root))
print(f"Number of Components: {network.n_components}")


N_ATTACKS = 1000 # Total number of attacks: no_of_components * N_ATTACKS
PARAM_NAME = "children_per_parent"
param_values = [2, 3, 5, 8, 13, 21, 34, network.n_devices]
analyzer = Analyzer(network)
# Estimated Time: 40m (6 core CPU)
compromised_array, effort_array, _ = analyzer.monte_carlo_multi_analysis(seed=seed, 
                                                                         n_attacks=N_ATTACKS,
                                                                         child_no_deviation=0,
                                                                         grid=grid,
                                                                         vary_entrypoints=True,
                                                                         param_name=PARAM_NAME, 
                                                                         param_values=param_values)
np.savez(p.cwd() / "data" / "Redundancy_NoSiblings.npz", compromise=compromised_array, effort=effort_array)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path as p
arrays = np.load(p.cwd() / "data" / "Redundancy_NoSiblings.npz")
compromised_array = arrays.get("compromise")
effort_array = arrays.get("effort")
PARAM_NAME = "children_per_parent"
param_values = [2, 3, 5, 8, 13, 21, 34, network.n_devices]

def plot_multi_monte(results, save_name="MultiMonte"):
    # Reshape to collapse all entrypoint variations into the first dimension
    n_attacks, n_entrypoints, n_params = results["compromised"].shape
    results["compromised"] = np.reshape(results["compromised"], (n_attacks*n_entrypoints, n_params))
    # Create Pandas Dataframe (to name attributes)
    df = pd.DataFrame(results["compromised"], columns=results["param_values"])
    df = df.melt(var_name=results["param_name"])

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
    sns.set_context(font_scale=2.0)
    sns.histplot(df, x="value", hue=results["param_name"], discrete=True, stat="probability", common_norm=False, ax=ax)
    sns.move_legend(ax, "upper right", ncols=4, title=" ".join([word.capitalize() for word in results["param_name"].split("_")]))
    ax.set(xlabel="No. of Devices Compromised", yscale="log")
    fig.savefig(p.cwd() / "media" / f"{save_name}.pdf")
    plt.show()
    
plot_multi_monte(results={"compromised":compromised_array, "effort":effort_array, "param_name":PARAM_NAME, "param_values":param_values}, save_name="Redundancy")

### Budget


In [ ]:
import random
import warnings
import numpy as np
import pandapower
from pathlib import Path as p

from communication.network import CommNetwork
from cyber.analysis import Analyzer
from attackers.random_attacker import RandomAttacker

seed = 42
np.random.seed(seed); random.seed(seed)
print(f"Seed: {seed}")

with warnings.catch_warnings():
    warnings.filterwarnings(action="ignore", category=FutureWarning)
    grid=pandapower.networks.create_cigre_network_mv(with_der="all")
    print(grid)
    spec_path = p.cwd() / "specifications" / "Default_specifications.json"
    network = CommNetwork(n_devices=3, n_entrypoints=1, children_per_parent=2, child_no_deviation=0, sibling_to_sibling_comm="all",
                          effort_only=True, network_specs=spec_path, grid=grid)
print(CommNetwork.show_tree(network.root))
print(f"Number of Components: {network.n_components}")


N_ATTACKS = 1000 # Total number of attacks: no_of_components * N_ATTACKS
PARAM_NAME = "budget"
param_values = [0.1, 1, 5, 10, 50, 100, 250, 500, 1000, 2500, 5000, 10000]
analyzer = Analyzer(network)
# Estimated Time: 37m (6 CPU cores)
compromised_array, effort_array, _ = analyzer.monte_carlo_multi_analysis(seed=seed, 
                                                                         n_attacks=N_ATTACKS,
                                                                         children_per_parent=3,
                                                                         child_no_deviation=0,
                                                                         grid=grid,
                                                                         effort_only=True,
                                                                         vary_entrypoints=True, 
                                                                         param_name=PARAM_NAME, 
                                                                         param_values=param_values)
np.savez(p.cwd() / "data" / "Budget_EffortOnly.npz", compromise=compromised_array, effort=effort_array)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path as p
arrays = np.load(p.cwd() / "data" / "Budget_EffortOnly.npz")
compromised_array = arrays.get("compromise")
effort_array = arrays.get("effort")
PARAM_NAME = "budget"
param_values = [0.1, 1, 5, 10, 50, 100, 250, 500, 1000, 2500, 5000, 10000]

plot_multi_monte(results={"compromised":compromised_array, "effort":effort_array, "param_name":PARAM_NAME, "param_values":param_values}, save_name="Budget_EffortOnly")

### Susceptibility